In [17]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [18]:
datos = pd.read_csv("train.csv", sep=",")
# Display de dataset
train = pd.DataFrame(datos)
train.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


## Manejo de datos faltantes




In [3]:
missing_values = train.isnull().sum()
print("Valores faltantes por columna:")
print(missing_values[missing_values > 0])

Valores faltantes por columna:
E_VALORMATRICULAUNIVERSIDAD     6287
E_HORASSEMANATRABAJA           30857
F_ESTRATOVIVIENDA              32137
F_TIENEINTERNET                26629
F_EDUCACIONPADRE               23178
F_TIENELAVADORA                39773
F_TIENEAUTOMOVIL               43623
E_PAGOMATRICULAPROPIO           6498
F_TIENECOMPUTADOR              38103
F_TIENEINTERNET.1              26629
F_EDUCACIONMADRE               23664
dtype: int64


In [4]:
# Impute missing values
for column in train.columns:
    if train[column].isnull().any():
        if train[column].dtype == 'object':
            # Impute categorical columns with mode
            mode_value = train[column].mode()[0]
            train[column] = train[column].fillna(mode_value)
        elif train[column].dtype in ['int64', 'float64']:
            # Impute numerical columns with median
            median_value = train[column].median()
            train[column] = train[column].fillna(median_value)

# Verify no remaining missing values
print("\nMissing values after imputation:")
print(train.isnull().sum())


Missing values after imputation:
ID                             0
PERIODO_ACADEMICO              0
E_PRGM_ACADEMICO               0
E_PRGM_DEPARTAMENTO            0
E_VALORMATRICULAUNIVERSIDAD    0
E_HORASSEMANATRABAJA           0
F_ESTRATOVIVIENDA              0
F_TIENEINTERNET                0
F_EDUCACIONPADRE               0
F_TIENELAVADORA                0
F_TIENEAUTOMOVIL               0
E_PRIVADO_LIBERTAD             0
E_PAGOMATRICULAPROPIO          0
F_TIENECOMPUTADOR              0
F_TIENEINTERNET.1              0
F_EDUCACIONMADRE               0
RENDIMIENTO_GLOBAL             0
INDICADOR_1                    0
INDICADOR_2                    0
INDICADOR_3                    0
INDICADOR_4                    0
dtype: int64


In [5]:

from sklearn.preprocessing import LabelEncoder


# 1. Separar las características (X) de la variable objetivo (y)
# X contendrá todas las columnas EXCEPTO la que queremos predecir.
X = train.drop('RENDIMIENTO_GLOBAL', axis=1)

# y contendrá SOLAMENTE la columna que queremos predecir.
y_texto = train['RENDIMIENTO_GLOBAL']

# 2. Instanciar el LabelEncoder
le = LabelEncoder()

# 3. Aplicar fit_transform a 'y'
# Esto aprende las categorías y las transforma en números
y = le.fit_transform(y_texto)


print("Mapeo de LabelEncoder:")
# Itera sobre las clases aprendidas y muestra su número
for i, clase in enumerate(le.classes_):
    print(f"{clase}  ->  {i}")

print("\nPrimeras 5 etiquetas en texto:")
print(y_texto.head())

print("\nPrimeras 5 etiquetas codificadas (como números):")
print(y[:5])

print(f"\nDimensiones de X (features): {X.shape}")
print(f"Dimensiones de y (target): {y.shape}")

Mapeo de LabelEncoder:
alto  ->  0
bajo  ->  1
medio-alto  ->  2
medio-bajo  ->  3

Primeras 5 etiquetas en texto:
0    medio-alto
1          bajo
2          bajo
3          alto
4    medio-bajo
Name: RENDIMIENTO_GLOBAL, dtype: object

Primeras 5 etiquetas codificadas (como números):
[2 1 1 0 3]

Dimensiones de X (features): (692500, 20)
Dimensiones de y (target): (692500,)


## Convertir variables categoricas one-hot




In [6]:
# Identificar las columnas categóricas excluyendo RENDIMIENTO_GLOBAL
categorical_cols = X.select_dtypes(include=['object']).columns
#aplicar get_dummies a las features
X_transformado = pd.get_dummies(X, columns=categorical_cols)

print(f"Dimensiones de X después de One-Hot Encoding: {X_transformado.shape}")

Dimensiones de X después de One-Hot Encoding: (692500, 1043)


In [7]:
print(X_transformado.shape)

(692500, 1043)


## Normalización


In [8]:
numerical_cols = X_transformado.select_dtypes(include=['int64', 'float64']).columns.tolist()
numerical_cols.remove('ID')
numerical_cols.remove('PERIODO_ACADEMICO')

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_transformado[numerical_cols] = scaler.fit_transform(X_transformado[numerical_cols])

display(X_transformado.head())

,ID,PERIODO_ACADEMICO,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4,E_PRGM_ACADEMICO_3° CICLO PROFESIONAL NEGOCIOS INTERNACIONALES,E_PRGM_ACADEMICO_ACTIVIDAD FISICA Y DEPORTE,E_PRGM_ACADEMICO_ACUICULTURA,E_PRGM_ACADEMICO_ADMINISTRACION,...,F_EDUCACIONMADRE_Ninguno,F_EDUCACIONMADRE_No Aplica,F_EDUCACIONMADRE_No sabe,F_EDUCACIONMADRE_Postgrado,F_EDUCACIONMADRE_Primaria completa,F_EDUCACIONMADRE_Primaria incompleta,F_EDUCACIONMADRE_Secundaria (Bachillerato) completa,F_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,F_EDUCACIONMADRE_Técnica o tecnológica completa,F_EDUCACIONMADRE_Técnica o tecnológica incompleta
0,904256,20212,0.437002,-0.556223,0.813978,0.060296,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
1,645256,20212,0.346934,-0.481341,0.508180,0.016142,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,308367,20203,0.232301,-0.492038,0.729034,0.016142,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
3,470353,20195,1.771650,-0.941332,-0.171371,-1.072993,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
4,989032,20212,0.387874,-0.299484,0.389259,0.457683,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False


In [19]:
display(X_transformado.head())
display(X_transformado.info())
display(X_transformado.describe())

,ID,PERIODO_ACADEMICO,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4,E_PRGM_ACADEMICO_3° CICLO PROFESIONAL NEGOCIOS INTERNACIONALES,E_PRGM_ACADEMICO_ACTIVIDAD FISICA Y DEPORTE,E_PRGM_ACADEMICO_ACUICULTURA,E_PRGM_ACADEMICO_ADMINISTRACION,...,F_EDUCACIONMADRE_Ninguno,F_EDUCACIONMADRE_No Aplica,F_EDUCACIONMADRE_No sabe,F_EDUCACIONMADRE_Postgrado,F_EDUCACIONMADRE_Primaria completa,F_EDUCACIONMADRE_Primaria incompleta,F_EDUCACIONMADRE_Secundaria (Bachillerato) completa,F_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,F_EDUCACIONMADRE_Técnica o tecnológica completa,F_EDUCACIONMADRE_Técnica o tecnológica incompleta
0,904256,20212,0.437002,-0.556223,0.813978,0.060296,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
1,645256,20212,0.346934,-0.481341,0.508180,0.016142,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,308367,20203,0.232301,-0.492038,0.729034,0.016142,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
3,470353,20195,1.771650,-0.941332,-0.171371,-1.072993,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
4,989032,20212,0.387874,-0.299484,0.389259,0.457683,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Columns: 1043 entries, ID to F_EDUCACIONMADRE_Técnica o tecnológica incompleta
dtypes: bool(1037), float64(4), int64(2)
memory usage: 716.6 MB


None

,ID,PERIODO_ACADEMICO,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
count,692500.000000,692500.000000,6.925000e+05,6.925000e+05,6.925000e+05,6.925000e+05
mean,494606.130576,20198.366679,-1.178731e-16,-1.508711e-16,2.909070e-16,6.960754e-17
std,285585.209455,10.535037,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00
min,1.000000,20183.000000,-2.199542e+00,-2.781294e+00,-4.452543e+00,-3.869422e+00
25%,247324.750000,20195.000000,-5.373728e-01,-5.134332e-01,-1.373936e-01,-1.163206e-01
50%,494564.500000,20195.000000,-2.344159e-01,1.177168e-01,2.363596e-01,3.252207e-01
75%,741782.500000,20203.000000,3.714978e-01,5.242203e-01,5.251688e-01,5.901454e-01
max,989286.000000,20213.000000,3.179990e+00,2.428368e+00,9.838658e-01,1.016969e+00


In [ ]:

# 1. Guardar X_transformado como Parquet
print("Guardando X_transformado en formato Parquet...")
X_transformado.to_parquet('X_train_procesado.parquet', index=False)
print("¡Listo!")

# 2. Guardar 'y'
import numpy as np
print("Guardando y_train...")
np.save('y_train_procesado.npy', y)
print("¡Listo!")


# MODELO

In [10]:
X_train_procesado = pd.read_parquet('X_train_procesado.parquet')
print("X_train_procesado cargado.")

X_train_procesado cargado.


In [11]:
import numpy as np
y_train_procesado = np.load('y_train_procesado.npy')
print("y_train_procesado cargado.")

y_train_procesado cargado.


In [12]:
# Celda añadida: verificación y alineamiento seguro de X_train_procesado y y_train_procesado
import numpy as np
import pandas as pd


# Asegurar que ambas variables existen antes de proceder
if 'X_train_procesado' in globals() and 'y_train_procesado' in globals():
    len_X = X_train_procesado.shape[0]
    try:
        len_y = y_train_procesado.shape[0]
    except Exception:
        len_y = len(y_train_procesado)

    if len_X != len_y:
        print(f'Inconsistencia detectada: len(X)={len_X}, len(y)={len_y}. Alineando de forma segura...')
        # Preferir eliminar entradas con y NaN si y es Series
        if isinstance(y_train_procesado, (pd.Series, pd.DataFrame)):
            mask = ~y_train_procesado.isnull()
            y_train_procesado = y_train_procesado.loc[mask].reset_index(drop=True)
            # Si X tiene al menos tantas filas como mask.sum(), aplicamos la misma máscara por posición
            if X_train_procesado.shape[0] >= mask.sum():
                X_train_procesado = X_train_procesado.iloc[:mask.sum()].reset_index(drop=True)
            else:
                X_train_procesado = X_train_procesado.iloc[:y_train_procesado.shape[0]].reset_index(drop=True)
        else:
            # y es numpy array: truncar el más largo para alinear por posición
            if len_X > len_y:
                X_train_procesado = X_train_procesado.iloc[:len_y].reset_index(drop=True)
            else:
                y_train_procesado = y_train_procesado[:len_X]
        print('Alineamiento completado. Nuevas longitudes:', X_train_procesado.shape[0], getattr(y_train_procesado, 'shape', len(y_train_procesado)))
    else:
        print('X e y ya estaban alineados. len=', len_X)
else:
    print('Aviso: X_train_procesado o y_train_procesado no están definidas en el kernel. Ejecuta las celdas de preprocesamiento apropiadas antes de entrenar.')

X e y ya estaban alineados. len= 692500


In [13]:
test_df = pd.read_csv('test.csv', sep=',')
print("test_df cargado.")

test_df cargado.


In [20]:
missing_values_test = test_df.isnull().sum()
print("Valores faltantes por columna en test_df:")
print(missing_values_test[missing_values_test > 0])

Valores faltantes por columna en test_df:
E_VALORMATRICULAUNIVERSIDAD     2723
E_HORASSEMANATRABAJA           13379
F_ESTRATOVIVIENDA              13795
F_TIENEINTERNET                11539
F_EDUCACIONPADRE                9993
F_TIENELAVADORA                17259
F_TIENEAUTOMOVIL               18918
E_PAGOMATRICULAPROPIO           2807
F_TIENECOMPUTADOR              16439
F_TIENEINTERNET.1              11539
F_EDUCACIONMADRE               10223
dtype: int64


In [21]:
for column in test_df.columns:
    if test_df[column].isnull().any():
        if test_df[column].dtype == 'object':
            mode_value = test_df[column].mode()[0]
            test_df[column] = test_df[column].fillna(mode_value)
        elif test_df[column].dtype in ['int64', 'float64']:
            median_value = test_df[column].median()
            test_df[column] = test_df[column].fillna(median_value)

print("Missing values in test_df after imputation:")
print(test_df.isnull().sum()[test_df.isnull().sum() > 0])

Missing values in test_df after imputation:
Series([], dtype: int64)


In [22]:
categorical_cols_test = test_df.select_dtypes(include=['object']).columns
X_test_transformed = pd.get_dummies(test_df, columns=categorical_cols_test)

print(f"Dimensiones de X_test_transformed después de One-Hot Encoding: {X_test_transformed.shape}")

Dimensiones de X_test_transformed después de One-Hot Encoding: (296786, 1014)


In [23]:
train_cols = X_train_procesado.columns
test_cols = X_test_transformed.columns

missing_in_test = set(train_cols) - set(test_cols)
for c in missing_in_test:
    X_test_transformed[c] = 0

missing_in_train = set(test_cols) - set(train_cols)
X_test_transformed = X_test_transformed.drop(columns=list(missing_in_train))

X_test_transformed = X_test_transformed[train_cols]

print(f"Dimensiones de X_test_transformed después de la alineación de columnas: {X_test_transformed.shape}")
print(f"Columnas de X_test_transformed son iguales a las de X_train_procesado: {all(X_test_transformed.columns == X_train_procesado.columns)}")

Dimensiones de X_test_transformed después de la alineación de columnas: (296786, 1039)
Columnas de X_test_transformed son iguales a las de X_train_procesado: True


In [24]:
from sklearn.model_selection import train_test_split

print("Librería train_test_split importada.")

Librería train_test_split importada.


In [25]:
X_train_sampled_eval, X_val_sampled_eval, y_train_sampled_eval, y_val_sampled_eval = train_test_split(
    X_train_procesado, y_train_procesado, test_size=0.8, stratify=y_train_procesado, random_state=42
)

print(f"Dimensiones de X_train_sampled_eval: {X_train_sampled_eval.shape}")
print(f"Dimensiones de y_train_sampled_eval: {y_train_sampled_eval.shape}")
print(f"Dimensiones de X_val_sampled_eval: {X_val_sampled_eval.shape}")
print(f"Dimensiones de y_val_sampled_eval: {y_val_sampled_eval.shape}")

Dimensiones de X_train_sampled_eval: (138500, 1039)
Dimensiones de y_train_sampled_eval: (138500,)
Dimensiones de X_val_sampled_eval: (554000, 1039)
Dimensiones de y_val_sampled_eval: (554000,)


In [26]:
import unicodedata
import re
import pandas as pd

def sanitize_and_deduplicate_cols(df):
    """
    Sanitiza nombres de columnas y maneja duplicados secuencialmente.
    """
    def sanitize(col):
        # 1. Normalizar a ASCII (elimina acentos y diéresis)
        new_col = unicodedata.normalize('NFKD', col).encode('ascii', 'ignore').decode('utf-8')
        
        # 2. Reemplazar caracteres especiales y espacios por guiones bajos
        new_col = re.sub(r'[^a-zA-Z0-9_]', '_', new_col)
        
        # 3. Eliminar guiones bajos duplicados o iniciales/finales
        new_col = re.sub(r'_+', '_', new_col).strip('_')
        
        # 4. LightGBM no le gustan los brackets: 
        new_col = new_col.replace('[', '_').replace(']', '_')
        
        return new_col

    sanitized_cols = [sanitize(col) for col in df.columns]
    
    # --- Manejo de Duplicados ---
    counts = {}
    final_cols = []
    for col in sanitized_cols:
        original_col = col
        if col in counts:
            counts[col] += 1
            # Añadir sufijo secuencial (e.g., _2, _3)
            col = f"{original_col}_{counts[original_col]}"
        else:
            counts[col] = 1
        final_cols.append(col)
        
    df.columns = final_cols
    return df

# --- Aplicar la limpieza a todos los conjuntos de datos de características ---

# 1. Aplicar a los datos de entrenamiento completo
X_train_procesado = sanitize_and_deduplicate_cols(X_train_procesado)

# 2. Aplicar a los conjuntos de tuning (evaluación rápida)
X_train_sampled_eval = sanitize_and_deduplicate_cols(X_train_sampled_eval)
X_val_sampled_eval = sanitize_and_deduplicate_cols(X_val_sampled_eval)

# 3. Aplicar al conjunto de prueba
X_test_transformed = sanitize_and_deduplicate_cols(X_test_transformed)

print("Nombres de columnas sanitizados y duplicados renombrados exitosamente.")
print(f"Número de columnas final en X_train: {X_train_procesado.shape[1]}")

Nombres de columnas sanitizados y duplicados renombrados exitosamente.
Número de columnas final en X_train: 1039


## Entrenar Modelo (RandomForestClassifier) con Datos Muestreados


Entrenar un modelo RandomForestClassifier utilizando el *conjunto de entrenamiento reducido* (X_train_sampled, y_train_sampled).


In [27]:
X_train_sampled, _, y_train_sampled, __ = train_test_split(X_train_procesado, y_train_procesado, test_size=0.9, stratify=y_train_procesado, random_state=42)

print(f"Dimensiones de X_train_sampled: {X_train_sampled.shape}")
print(f"Dimensiones de y_train_sampled: {y_train_sampled.shape}")

Dimensiones de X_train_sampled: (69250, 1039)
Dimensiones de y_train_sampled: (69250,)


In [28]:
from sklearn.ensemble import RandomForestClassifier

print("Entrenando RandomForestClassifier con datos muestreados...")
model_sampled = RandomForestClassifier(random_state=42)
model_sampled.fit(X_train_sampled, y_train_sampled)

print("¡Modelo entrenado exitosamente con datos muestreados!")

Entrenando RandomForestClassifier con datos muestreados...
¡Modelo entrenado exitosamente con datos muestreados!


In [29]:
print("Realizando predicciones en el conjunto de validación muestreado...")
y_pred_val = model_sampled.predict(X_val_sampled_eval)

print("¡Predicciones realizadas exitosamente en el conjunto de validación!")
print(f"Primeras 5 predicciones en validación: {y_pred_val[:5]}")

Realizando predicciones en el conjunto de validación muestreado...
¡Predicciones realizadas exitosamente en el conjunto de validación!
Primeras 5 predicciones en validación: [1 2 1 3 2]


In [30]:
from sklearn.metrics import accuracy_score, f1_score

print("Calculando métricas de evaluación...")

accuracy = accuracy_score(y_val_sampled_eval, y_pred_val)
f1 = f1_score(y_val_sampled_eval, y_pred_val, average='weighted')

print(f"Precisión (Accuracy) en el conjunto de validación: {accuracy:.4f}")
print(f"F1-Score ponderado en el conjunto de validación: {f1:.4f}")

Calculando métricas de evaluación...
Precisión (Accuracy) en el conjunto de validación: 0.3944
F1-Score ponderado en el conjunto de validación: 0.3835


In [31]:
print("Realizando predicciones en los datos de prueba...")
predictions_encoded = model_sampled.predict(X_test_transformed)

print("¡Predicciones realizadas exitosamente!")
print(f"Primeras 5 predicciones codificadas: {predictions_encoded[:5]}")

Realizando predicciones en los datos de prueba...
¡Predicciones realizadas exitosamente!
Primeras 5 predicciones codificadas: [1 0 0 1 0]


In [32]:
print("Inverse transformando las predicciones a etiquetas originales...")
predictions_decoded = le.inverse_transform(predictions_encoded)

print("¡Predicciones decodificadas exitosamente!")
print(f"Primeras 5 predicciones decodificadas: {predictions_decoded[:5]}")

Inverse transformando las predicciones a etiquetas originales...
¡Predicciones decodificadas exitosamente!
Primeras 5 predicciones decodificadas: ['bajo' 'alto' 'alto' 'bajo' 'alto']


In [33]:
print("Creando el archivo de envío (submission.csv)...")
submission_df = pd.DataFrame({'ID': test_df['ID'], 'RENDIMIENTO_GLOBAL': predictions_decoded})

submission_df.to_csv('submission(7).csv', index=False)

print("¡Archivo submission.csv creado exitosamente!")
print("Primeras 5 filas del archivo de envío:")
print(submission_df.head())

Creando el archivo de envío (submission.csv)...
¡Archivo submission.csv creado exitosamente!
Primeras 5 filas del archivo de envío:
       ID RENDIMIENTO_GLOBAL
0  550236               bajo
1   98545               alto
2  499179               alto
3  782980               bajo
4  785185               alto


# Tunning

In [34]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

print("Importando librerías para RandomizedSearchCV...")

# 2. Definir el espacio de búsqueda de hiperparámetros
param_dist = {
    'n_estimators': [50, 100, 150],
    'max_depth': [10, 20, 30, None],
    'min_samples_leaf': [1, 2, 4]
}

print("Espacio de hiperparámetros definido.")

# 3. Instanciar un RandomForestClassifier
rf = RandomForestClassifier(random_state=42)

# 4. Instanciar RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=10,  # Número de configuraciones de parámetros a probar
    cv=3,       # 3-fold cross-validation
    scoring='f1_weighted', # Usar f1_weighted como métrica de evaluación
    random_state=42,
    n_jobs=-1   # Usar todos los procesadores disponibles
)

print("RandomizedSearchCV instanciado. Iniciando búsqueda...")

# 5. Ajustar RandomizedSearchCV a los datos de entrenamiento muestreados
random_search.fit(X_train_sampled_eval, y_train_sampled_eval)

print("Búsqueda de hiperparámetros completada.")

# 6. Imprimir los mejores parámetros encontrados
print("Mejores parámetros encontrados:")
print(random_search.best_params_)

# 7. Imprimir el mejor F1-score alcanzado
print("Mejor F1-Score (ponderado) encontrado:")
print(random_search.best_score_)

Importando librerías para RandomizedSearchCV...
Espacio de hiperparámetros definido.
RandomizedSearchCV instanciado. Iniciando búsqueda...
Búsqueda de hiperparámetros completada.
Mejores parámetros encontrados:
{'n_estimators': 100, 'min_samples_leaf': 2, 'max_depth': None}
Mejor F1-Score (ponderado) encontrado:
0.38938201988231524


In [35]:
print("Obteniendo los mejores hiperparámetros...")
best_params = random_search.best_params_

print("Instanciando RandomForestClassifier con los mejores hiperparámetros...")
model_full_data = RandomForestClassifier(**best_params, random_state=42)

print("Entrenando el modelo con el conjunto de datos de entrenamiento completo...")
model_full_data.fit(X_train_procesado, y_train_procesado)

print("¡Modelo entrenado exitosamente con los mejores hiperparámetros en los datos completos!")

Obteniendo los mejores hiperparámetros...
Instanciando RandomForestClassifier con los mejores hiperparámetros...
Entrenando el modelo con el conjunto de datos de entrenamiento completo...
¡Modelo entrenado exitosamente con los mejores hiperparámetros en los datos completos!


In [36]:
print("Realizando predicciones en los datos de prueba con el modelo de datos completos...")
predictions_full_data_encoded = model_full_data.predict(X_test_transformed)

print("¡Predicciones realizadas exitosamente con el modelo de datos completos!")
print(f"Primeras 5 predicciones codificadas del modelo completo: {predictions_full_data_encoded[:5]}")

Realizando predicciones en los datos de prueba con el modelo de datos completos...
¡Predicciones realizadas exitosamente con el modelo de datos completos!
Primeras 5 predicciones codificadas del modelo completo: [1 1 1 1 1]


In [37]:
print("Inverse transformando las predicciones a etiquetas originales...")
predictions_final_decoded = le.inverse_transform(predictions_full_data_encoded)

print("¡Predicciones decodificadas exitosamente!")
print(f"Primeras 5 predicciones decodificadas: {predictions_final_decoded[:5]}")

Inverse transformando las predicciones a etiquetas originales...
¡Predicciones decodificadas exitosamente!
Primeras 5 predicciones decodificadas: ['bajo' 'bajo' 'bajo' 'bajo' 'bajo']


In [38]:
print("Creando el archivo de envío (submission.csv)...")
submission_df = pd.DataFrame({'ID': test_df['ID'], 'RENDIMIENTO_GLOBAL': predictions_final_decoded})

submission_df.to_csv('submission (6).csv', index=False)

print("¡Archivo submission.csv creado exitosamente!")
print("Primeras 5 filas del archivo de envío:")
print(submission_df.head())

Creando el archivo de envío (submission.csv)...
¡Archivo submission.csv creado exitosamente!
Primeras 5 filas del archivo de envío:
       ID RENDIMIENTO_GLOBAL
0  550236               bajo
1   98545               bajo
2  499179               bajo
3  782980               bajo
4  785185               bajo
